# template construction:
1. Make a directory for your data. Copy or link all the images into it.
2. On the command line, within that directory, run the following command: bash
antsMultivariateTemplateConstruction2.sh to get usage.
### **Example: **
antsMultivariateTemplateConstruction.sh -d 3 -m 30x50x20 -t GR -s CC -c 1 -o MY -z InitialTemplate.nii.gz  *RF*T1x.nii.gz

- In this example 30x50x20 iterations per registration are used for template creation (that is the default)
- Greedy-SyN and CC are the metrics to guide the mapping.
- Output is prepended with MY and the initial template is InitialTemplate.nii.gz (optional).
- The -c option is set to 1, which will result in using the Sun Grid Engine (SGE) to distribute the computation.
- if you do not have SGE, read the help for multi-core computation on the local machine, or Apple X-grid options.

-----------------------------------------------------------
Script written by Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

**Creat a folder for template construction and copy all anatoical data to this folder**

In [ ]:
import os
import glob
from shutil import copyfile, rmtree

data_path = '/mnt/hgfs/D/Data/Pigeon/Sleep/analysis'
template_path = '/mnt/hgfs/D/Data/Pigeon/Sleep/Scripts/template'
anat_dirs = glob.glob('%s/sub*/*/t2w.nii.gz'%(data_path))
 # creat a template directory
if os.path.exists(template_path):
    print('Tempelate directory already exist!')
    pass
else:
    print('Creating template directory ...')
    os.mkdir(template_path)
    
for curr_anat in anat_dirs:
    print('Copying %s >>> %s \n'%(curr_anat,template_path))
    curr_anat_split = curr_anat.split('/');
    sub_name = curr_anat_split[-3]+'_'+curr_anat_split[-2]+'.nii.gz'
          
    dst = os.path.join(template_path,sub_name)
    copyfile(curr_anat,dst)

### **run antsMultivariateTemplateConstruction**

In [ ]:
import os
os.chdir(template_path)
os.system('antsMultivariateTemplateConstruction.sh -d 3 -m 30x50x20 -t GR -s CC -c 0 -z sub29_20191022.nii.gz -o TEMP sub*.nii.gz')

### rename file

In [ ]:
# rename file
os.system('fslmaths TEMPtemplate0.nii.gz standard_head.nii.gz')

### remove non related fiels from temp directory

In [ ]:
os.system('rm -r TEMP* inter* sub*')

### resample the template to have 2mm isotropic resolution

In [ ]:
os.system('flirt -in standard_head -ref standard_head -out standard_head -applyisoxfm 1.95')

### **clean the template manually**

In [ ]:
os.chdir(template_path)
# extract clear brain 
os.system('bet standard_head standard_brain -R -f 0.4 -g 0 -m')



### apply clean mask

In [ ]:
os.system('fslmaths standard_head -mul standard_brain_mask.nii.gz standard_brain')